In [1]:
import torch as th
from torch import nn
import numpy as np
import torchvision as thv
import time

In [2]:
DEVICE = "cuda" if th.cuda.is_available() else "cpu"
DEVICE

'cuda'

In [3]:
train_dataset = thv.datasets.CIFAR10(root="E:\PyTorch\Working Data\Input Data",
                                     train=True,
                                     download=True,
                                     transform=thv.transforms.ToTensor())

Files already downloaded and verified


In [4]:
test_dataset = thv.datasets.CIFAR10(root="E:\PyTorch\Working Data\Input Data",
                                    train=False,
                                    download=True,
                                    transform=thv.transforms.ToTensor())

Files already downloaded and verified


In [5]:
X_train = [x for x,y in train_dataset]

In [6]:
Y_train = [[y] for x,y in train_dataset]

In [7]:
X_test = [x for x,y in test_dataset]

In [8]:
Y_test = [[y] for x,y in test_dataset]

In [9]:
from sklearn.preprocessing import OneHotEncoder

OHE = OneHotEncoder(sparse_output=False)

Y_main_train = OHE.fit_transform(Y_train)
Y_main_test = OHE.transform(Y_test)

In [10]:
X_train_tensor = th.Tensor(np.array(X_train))
Y_train_tensor = th.Tensor(Y_main_train)
X_test_tensor = th.Tensor(np.array(X_test))
Y_test_tensor = th.Tensor(Y_main_test)

In [11]:
train_main_dataset = th.utils.data.TensorDataset(X_train_tensor,Y_train_tensor)

In [12]:
train_main_dataset[0]

(tensor([[[0.2314, 0.1686, 0.1961,  ..., 0.6196, 0.5961, 0.5804],
          [0.0627, 0.0000, 0.0706,  ..., 0.4824, 0.4667, 0.4784],
          [0.0980, 0.0627, 0.1922,  ..., 0.4627, 0.4706, 0.4275],
          ...,
          [0.8157, 0.7882, 0.7765,  ..., 0.6275, 0.2196, 0.2078],
          [0.7059, 0.6784, 0.7294,  ..., 0.7216, 0.3804, 0.3255],
          [0.6941, 0.6588, 0.7020,  ..., 0.8471, 0.5922, 0.4824]],
 
         [[0.2431, 0.1804, 0.1882,  ..., 0.5176, 0.4902, 0.4863],
          [0.0784, 0.0000, 0.0314,  ..., 0.3451, 0.3255, 0.3412],
          [0.0941, 0.0275, 0.1059,  ..., 0.3294, 0.3294, 0.2863],
          ...,
          [0.6667, 0.6000, 0.6314,  ..., 0.5216, 0.1216, 0.1333],
          [0.5451, 0.4824, 0.5647,  ..., 0.5804, 0.2431, 0.2078],
          [0.5647, 0.5059, 0.5569,  ..., 0.7216, 0.4627, 0.3608]],
 
         [[0.2471, 0.1765, 0.1686,  ..., 0.4235, 0.4000, 0.4039],
          [0.0784, 0.0000, 0.0000,  ..., 0.2157, 0.1961, 0.2235],
          [0.0824, 0.0000, 0.0314,  ...,

In [13]:
test_main_dataset[0]

NameError: name 'test_main_dataset' is not defined

In [14]:
test_main_dataset = th.utils.data.TensorDataset(X_test_tensor,Y_test_tensor)

In [15]:
train_dataloader = th.utils.data.DataLoader(train_main_dataset,
                                            batch_size=32,
                                            shuffle=True)

In [16]:
test_dataloader = th.utils.data.DataLoader(test_main_dataset,
                                           batch_size=32, 
                                           shuffle=False)

In [17]:
model = nn.Sequential(
                        nn.Conv2d(in_channels=3,out_channels=10,kernel_size=3,stride=1,padding="valid"),
                        nn.ReLU(),
                        nn.Conv2d(in_channels=10,out_channels=10,kernel_size=3,stride=1,padding="valid"),
                        nn.ReLU(),
                        nn.MaxPool2d(kernel_size=2),
                        nn.Conv2d(in_channels=10,out_channels=10,kernel_size=3,stride=1,padding="valid"),
                        nn.ReLU(),
                        nn.Conv2d(in_channels=10,out_channels=10,kernel_size=3,stride=1,padding="valid"),
                        nn.ReLU(),
                        nn.MaxPool2d(kernel_size=2),
                        nn.Flatten(),
                        nn.Linear(in_features=250,out_features=10)
)

In [18]:
trainable_params = sum(
    p.numel() for p in model.parameters()
)
trainable_params

5520

In [19]:
loss_fn = nn.CrossEntropyLoss()

optimizer = th.optim.Adam(model.parameters(),
                          lr=0.001)

In [20]:
def accuracy(Y_pred,Y_true):
    correct = th.eq(Y_pred,Y_true).sum().item()
    acc = (correct/len(Y_pred))*100
    return acc

In [21]:
TRAIN_DL_LEN = len(train_dataloader)
TEST_DL_LEN = len(test_dataloader)
EPOCHS = 1

model.to(DEVICE)
for epoch in range(EPOCHS):
    model.train()
    epoch_train_loss = 0.0
    epoch_train_accuracy = 0.0
    for X,Y in train_dataloader:
        X = X.to(DEVICE)
        Y = Y.to(DEVICE)

        start = time.time()
        Y_logits = model(X).squeeze()
        end = time.time()
        print(f"Model Train Time {end - start}")
        Y_true = th.argmax(Y,1)
        Y_pred = th.argmax(th.softmax(Y_logits,1),1)
        
        loss = loss_fn(Y_logits,Y)
        acc = accuracy(Y_pred,Y_true)
        
        epoch_train_loss += loss
        epoch_train_accuracy += acc
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    model.eval()
    epoch_test_loss = 0.0
    epoch_test_accuracy = 0.0
    with th.no_grad():
        for X,Y in test_dataloader:
            X = X.to(DEVICE)
            Y = Y.to(DEVICE)

            start = time.time()
            Y_logits = model(X).squeeze()
            end = time.time()
            print(f"Infernce Time {end - start}")
            Y_true = th.argmax(Y,1)
            Y_pred = th.argmax(th.softmax(Y_logits,1),1)
            
            loss = loss_fn(Y_logits,Y)
            acc = accuracy(Y_pred,Y_true)
            
            epoch_test_accuracy += acc
            epoch_test_loss += loss
            
    
    epoch_train_accuracy /= TRAIN_DL_LEN
    epoch_train_loss /= TRAIN_DL_LEN
    epoch_test_accuracy /= TEST_DL_LEN
    epoch_test_loss /= TEST_DL_LEN
    end=time.time()
    
    print(f"Epoch {epoch}[{round(end-start,2)}] :- |Train Loss:- {epoch_train_loss:.2f} <--------> Train Accuracy:- {epoch_train_accuracy:.2f}|++++++++|Test Loss:- {epoch_test_loss:.2f} <--------> Test Accuracy :- {epoch_test_accuracy:.2f}|")

Model Train Time 0.24235296249389648
Model Train Time 0.000997304916381836
Model Train Time 0.0009970664978027344
Model Train Time 0.0
Model Train Time 0.0009963512420654297
Model Train Time 0.0009980201721191406
Model Train Time 0.0
Model Train Time 0.0009975433349609375
Model Train Time 0.0
Model Train Time 0.0
Model Train Time 0.000997781753540039
Model Train Time 0.0
Model Train Time 0.0009970664978027344
Model Train Time 0.0
Model Train Time 0.000997781753540039
Model Train Time 0.0
Model Train Time 0.0009968280792236328
Model Train Time 0.000997304916381836
Model Train Time 0.0
Model Train Time 0.000997781753540039
Model Train Time 0.0009970664978027344
Model Train Time 0.0009980201721191406
Model Train Time 0.0
Model Train Time 0.0009975433349609375
Model Train Time 0.0
Model Train Time 0.0
Model Train Time 0.0009975433349609375
Model Train Time 0.0009975433349609375
Model Train Time 0.0009984970092773438
Model Train Time 0.0
Model Train Time 0.0
Model Train Time 0.0009975433349